In [1]:
import pygmo as pg
import numpy as np
from problem import SignalProcessingParams

In [2]:
prob1 = pg.problem(SignalProcessingParams(group_number=1))
# prob2 = pg.problem(SignalProcessingParams(group_number=2))
print(prob1)
# print(prob2)

Problem name: Optimize EMG signal processing parameters to cross-corelate with grip force
	C++ class name: pybind11::object

	Global dimension:			251
	Integer dimension:			1
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [0, 0, 0, 0, 0, ... ]
	Upper bounds: [2, 2, 2, 2, 2, ... ]
	Has batch fitness evaluation: false

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 0

	Thread safety: none

Extra info:
	Sampling rate: 992.969696969697 Hz
	FFT window size: 496 samples
	FFT resolution: 2.0019550342130987 Hz



In [3]:
# Add parameter names using FFT frequencies
param_names = list(np.round(prob1.extract(SignalProcessingParams).emg_fft_freq[1:], decimals=2))
param_names = ['f' + str(i) for i in param_names] + ['decay', 'window']
param_names = [i.replace('.', '_') for i in param_names]
np.savetxt('param_names.txt', X=np.column_stack((
  np.arange(start=1, stop=len(param_names)+1), param_names
  )), fmt='%s')
# param_names

In [5]:
# params_select = np.loadtxt('emg_dsample.txt', delimiter=' ')
# params_select = np.loadtxt('emg_lvct_select_11.txt', delimiter=' ')
params_select = np.loadtxt('emg_lvct_select_5.txt', delimiter=' ')
display(params_select.shape)
# selected_params = ('f2_0', 'f4_0', 'f50_05', 'f120_12', 'f124_12', 'f136_13', 'f176_17',
#                    'f414_4', 'f472_46', 'decay', 'window')
selected_params = ('f2_0', 'f4_0', 'f50_05', 'decay', 'window')
# np.savetxt('param_names_select_5.txt', X=np.column_stack((
#   np.arange(start=1, stop=len(selected_params)+1), selected_params
#   )), fmt='%s')
selected_index = [param_names.index(i) for i in selected_params]
display(selected_index)
# Always remove DC offset
params = np.ones((params_select.shape[0], 250))
params[:, selected_index] = params_select
# params = params_select
params = np.column_stack((np.zeros(params.shape[0]), params))
params.shape

(2000, 5)

[0, 1, 24, 248, 249]

(2000, 251)

In [ ]:
corr1 = [1 - prob1.fitness(i) for i in params]
# corr2 = [prob2.fitness(i) for i in params]

In [ ]:
min(corr1)
corr2 = np.loadtxt('corr2.csv')
display(corr2[:10])
np.savetxt('corrs_dsample.txt', X=np.column_stack((corr1, corr2)), delimiter=' ')

In [ ]:
# Slicing only first 100 corrs and params
np.savetxt('emg_dsample_100.txt', X=params[:100, 1:], delimiter=' ')
np.savetxt('corrs_dsample_100.txt', X=np.column_stack((corr1, corr2))[:100, :], delimiter=' ')

In [ ]:
%%timeit -n1 -r1
pop1 = pg.population(prob, size=20)

In [ ]:
3300 / 3600

In [ ]:
print(pop1)

In [ ]:
pop1.get_x()[32][-1]

In [ ]:
pop1.champion_x